In [1]:
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import datasets
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from math import sqrt
# from pprint import pprint
# from sklearn.model_selection import RandomizedSearchCV
# import pickle

# Load Data

In [52]:
df = pd.read_csv('/Users/zhangruiqi/Downloads/TCD/Modules/CS7CS4ML/IndividualProject/tcdml1920-income-ind/tcd ml 2019-20 income prediction training (with labels).csv')

In [53]:
def fill_null(df):
    for i in df:
        if df[i].dtype.kind in 'biufc':
            df[i] = df[i].replace(np.nan,0)
        else:
            df[i] = df[i].replace(np.nan,'0')
    return df

def filter_zero(df):
    df['isValuable'] = 'Yes'
    for i in df:
        if df[i].dtype.kind in 'biufc':
            df['isValuable'] = np.where(df[i] == 0, 'No' ,df['isValuable'])
        if df[i].dtype.kind not in 'biufc':
            df['isValuable'] = np.where(df[i] == '0', 'No' ,df['isValuable'])
    return df

In [54]:
df = fill_null(df)

# Remove possible noises
df['Hair Color'] = np.where(df['Hair Color'] == 'unknown', '0',df['Hair Color'])
df['Gender'] = np.where(df['Gender'] == 'unknown', '0', np.where(df['Gender'] == 'other', '0', df['Gender']))
df['University Degree'] = np.where(df['University Degree'] == '0', 'No', df['University Degree'])
# impute data 0s so that they won't be filtered out
df['Wears Glasses'] = df['Wears Glasses'].apply(str)
df['Wears Glasses'] = np.where(df['Wears Glasses'] == '0', 'No',np.where(df['Wears Glasses'] == '1', 'Yes',df['Wears Glasses']))

df = filter_zero(df)

In [55]:
df['University Degree'].value_counts()

Bachelor    43172
No          35059
Master      26941
PhD          6821
Name: University Degree, dtype: int64

In [56]:
df.tail()

,Instance,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR,isValuable
111988,111989,1980.0,female,29.0,Honduras,205274,industrial program compliance analyst,No,No,Brown,135,17185.1297,Yes
111989,111990,2006.0,female,36.0,Tunisia,1012826,senior policy advisor,Master,Yes,Brown,147,118806.5392,Yes
111990,111991,1993.0,female,36.0,Sri Lanka,60848,messenger,Master,Yes,Brown,153,289951.3294,Yes
111991,111992,2019.0,0,54.0,Denmark,848640,senior case support associate,No,No,Black,190,100046.5278,No
111992,111993,2017.0,female,27.0,State of Palestine,325590,project manager,No,No,Blond,174,145886.2885,Yes


In [28]:
# cat_cols = df.select_dtypes(include = 'object').columns[:-1]
# cat_cols

Index(['Gender', 'Country', 'Profession', 'University Degree', 'Wears Glasses',
       'Hair Color'],
      dtype='object')

In [57]:
df2 = df.copy()
cat_cols = df.select_dtypes(include = 'object').columns[:-1]

for i in cat_cols:
    mean_val = df2.groupby(i)['Income in EUR'].mean()
    df2.loc[:, i] = df2[i].map(mean_val)

df2 = df2[df2['isValuable'] == 'Yes']

df2 = df2.drop(columns = ['isValuable','Instance'])
df2

,Year of Record,Gender,Age,Country,Size of City,Profession,University Degree,Wears Glasses,Hair Color,Body Height [cm],Income in EUR
4,2010.0,99913.787444,46.0,42773.138452,95389,88209.535957,100028.890970,108356.382469,109119.903312,188,38022.16217
5,1991.0,112426.895003,60.0,115542.390790,56080,123970.433478,108933.097570,108356.382469,109119.903312,181,125809.99030
7,2008.0,112426.895003,36.0,26046.759744,31419,114689.720790,100028.890970,108356.382469,110060.140156,190,11849.02876
8,2015.0,112426.895003,43.0,84512.173088,85333,58086.623603,100028.890970,108356.382469,109119.903312,189,48058.20607
9,2019.0,99913.787444,16.0,100607.155049,1325383,104474.914110,108933.097570,110069.412884,108685.538142,157,90940.68912
...,...,...,...,...,...,...,...,...,...,...,...
111987,2008.0,112426.895003,47.0,62039.079598,844634,138217.951178,108933.097570,108356.382469,108685.538142,171,78834.40320
111988,1980.0,99913.787444,29.0,40478.734970,205274,59594.668358,100028.890970,108356.382469,110060.140156,135,17185.12970
111989,2006.0,99913.787444,36.0,135724.673788,1012826,135245.179782,116507.320667,110069.412884,110060.140156,147,118806.53920
111990,1993.0,99913.787444,36.0,429979.701959,60848,98273.245191,116507.320667,110069.412884,110060.140156,153,289951.32940


In [58]:
Y = df2['Income in EUR']
X = df2.drop('Income in EUR', axis = 1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)
rfs = RandomForestRegressor(n_estimators=100)
rfs = rfs.fit(X_train, Y_train)
rfs.score(X_test,Y_test)

0.8412331831541475

In [63]:
dfSubmission = pd.read_csv('/Users/zhangruiqi/Downloads/TCD/Modules/CS7CS4ML/IndividualProject/tcdml1920-income-ind/tcd ml 2019-20 income prediction test (without labels).csv')

dfSubmission = fill_null(dfSubmission)

dfSubmission['Hair Color'] = np.where(dfSubmission['Hair Color'] == 'unknown', '0',dfSubmission['Hair Color'])
dfSubmission['Gender'] = np.where(dfSubmission['Gender'] == 'unknown', '0',np.where(dfSubmission['Gender'] == 'other', '0', dfSubmission['Gender']))
df['University Degree'] = np.where(df['University Degree'] == '0', 'No', df['University Degree'])


dfSubmission['Wears Glasses'] = dfSubmission['Wears Glasses'].apply(str)
dfSubmission['Wears Glasses'] = np.where(dfSubmission['Wears Glasses'] == '0', 'No',np.where(dfSubmission['Wears Glasses'] == '1', 'Yes',dfSubmission['Wears Glasses']))

dfSubmission = filter_zero(dfSubmission)
dfSubmission2 = dfSubmission.copy()

cat_cols = dfSubmission2.select_dtypes(include = 'object').columns[:-1]

for i in cat_cols:
    mean_val = df.groupby(i)['Income in EUR'].mean()
    dfSubmission2.loc[:, i] = dfSubmission2[i].map(mean_val)

dfSubmission2 = dfSubmission2.drop(['isValuable','Instance'], axis = 1)
dfSubmission2.head()

x = dfSubmission2.columns[:-1]
x_submission = dfSubmission2[x]

x_submission = x_submission.fillna(0)

x_submission['Income in EUR'] = rfs.predict(x_submission)

# x_submission.to_csv('test.csv')

# submission['Income'] = y_Pred
# submission.to_csv(submissionFile, index=False)

In [64]:
submissionFile = '/Users/zhangruiqi/Downloads/TCD/Modules/CS7CS4ML/IndividualProject/tcdml1920-income-ind/tcd ml 2019-20 income prediction submission file.csv'
submission = pd.read_csv(submissionFile)
tmp = x_submission['Income in EUR']

In [65]:
submission['Income'] = tmp

In [66]:
submission.to_csv(submissionFile, index=False)